# Symbolic Computation with SymPy
Symbolic computation deals with the computation of mathematical objects symbolically. This means that the mathematical objects are represented exactly, not approximately, and mathematical expressions with unevaluated variables are left in symbolic form.

In [23]:
from sympy import *

Here is a brief example, if you wanted to compute a square root in python you would do:

In [13]:
import math
math.sqrt(8)

2.8284271247461903

This is an approximate solution because $\sqrt{8}$ is an irrational number. But if you were to use the sympy version of of the square root function:

In [24]:
sqrt(8)

2*sqrt(2)

Notice that this is an exact solution, where the irrational part is left unevaluated by default.

Let's move to a more interesting examples

---
## The power of symbolic computation

In [25]:
x, y = symbols('x y')
type(x)

sympy.core.symbol.Symbol

In [26]:
y

y

Now the code understands that `x` and `y` are symbols, not variables. These symbols can be used to build expressions:

In [27]:
expr = x + 2*y
expr

x + 2*y

In [28]:
expr + 1

x + 2*y + 1

In [29]:
(x**2)*expr

x**2*(x + 2*y)

Notice that in this last expression, the $x^2$ term was not distributed, but that can be requested.

In [30]:
expand((x**2)*expr)

x**3 + 2*x**2*y

The reverse operation of `expand` is `factor`:

In [31]:
factor(x**2 + x)

x*(x + 1)

Here are a few more interesting examples:

In [32]:
sp.diff(x*sin(x), x)

x*cos(x) + sin(x)

In [33]:
sp.integrate(x*cos(x) + sin(x), x)

x*sin(x)

In [34]:
sp.integrate(sin(x**2), (x, -oo, oo))

sqrt(2)*sqrt(pi)/2

In [35]:
sp.limit(sin(x)/x, x, 0)

1

In [36]:
sp.solve(x**2-4, x)

[-2, 2]

In [37]:
t = symbols('t')
y = Function('y')
ode = dsolve(Eq(y(t).diff(t, t) - y(t), exp(t)), y(t))
ode

Eq(y(t), C2*exp(-t) + (C1 + t/2)*exp(t))

In [38]:
latex(ode)

'y{\\left(t \\right)} = C_{2} e^{- t} + \\left(C_{1} + \\frac{t}{2}\\right) e^{t}'

---
## Symbols VS Variables

Let's understand a bit of details of SymPy's implementation.

`symbols` takes a string of variable names separated by spaces or commas, and creates Symbols out of them. We can then assign these to variable names.

In [39]:
x, y, z = symbols('x, y, z')

The important thing to notice here is that the name of a Symbol and the name of the variable it is assigned to need not have anything to do with one another. In other words, **the variable and the symbol are not the same thing**.

In [40]:
a, b, c = symbols('x, y, z')

In [41]:
a

x

In [42]:
crazy = symbols('unralated')
crazy + 1

unralated + 1

Take the following example. Changing `x` to 2 had no effect on `expr`. This is because `x = 2` changes the Python variable `x` to 2, but has no effect on the SymPy Symbol x, which was what we used in creating `expr`.

In [43]:
x = symbols('x')
expr = x + 1
x = 2
display(expr)

x + 1

---
## Basic Operations
Here we discuss some of the most basic operations needed for expression manipulation in SymPy.

In [44]:
x, y, z = symbols("x y z")

One of the most common things you might want to do with a mathematical expression is **substitution**. Substitution replaces all instances of something in an expression with something else. It is done using the `subs` method. For example:

In [45]:
expr = cos(x) + 1
expr.subs(x, y)

cos(y) + 1

In [46]:
expr.subs(x, 0)

2

In [47]:
expr.subs(x, 2)

cos(2) + 1

Notice that this last example the value of the expression was not evaluated. That is because $\cos\left(2\right)$ is irrational (proof [here](https://math.stackexchange.com/questions/94478/sin-1-circ-is-irrational-but-how-do-i-prove-it-in-a-slick-way-and-tan1)). If we wanted to evaluate the expression, we would use the `evalf` method.

In [48]:
expr.subs(x, 2).evalf()

0.583853163452858

Notice that the output has the traditional 15 decimal places. But the `evalf` method lets you choose how many digits you want.

In [49]:
expr.subs(x, 2).evalf(30)

0.583853163452857613002431770499

Take a look at the first 100 digits of $\pi$

In [50]:
pi.evalf(100)

3.141592653589793238462643383279502884197169399375105820974944592307816406286208998628034825342117068

To perform multiple substitutions at once, pass a list of `(old, new)` pairs to `subs`.

In [51]:
expr = x**3 + 4*x*y - z
expr.subs([(x, 2), (y, 4), (z, 0)])

40

You can use python's list comprehensions to generate substitutions. In the example below we replace all instances of $x$ that have an even power with $y$.

In [52]:
expr = x**4 - 4*x**3 + 4*x**2 - 2*x + 3
replacements = [(x**i, y**i) for i in range(5) if i % 2 == 0]
expr.subs(replacements)

-4*x**3 - 2*x + y**4 + 4*y**2 + 3

---
## Printing
As you may have noticed, the output of symbolyc expressions in the jupyter notebook environment automatically prints the rendered $\LaTeX$ expression. But you can print outputs in other formats as well.

In [53]:
expr = Integral(sqrt(1/x), x)
expr

Integral(sqrt(1/x), x)

You can ask for the latex source code.

In [54]:
latex(expr)

'\\int \\sqrt{\\frac{1}{x}}\\, dx'

Use the `pprint` to print the expression using Unicode characters.

In [55]:
pprint(expr)

⌠           
⎮     ___   
⎮    ╱ 1    
⎮   ╱  ─  dx
⎮ ╲╱   x    
⌡           


Or non-unicode charcters

In [56]:
pprint(expr, use_unicode=False)

  /          
 |           
 |     ___   
 |    / 1    
 |   /  -  dx
 | \/   x    
 |           
/            


---
## Simplification
One of the most useful features of a symbolic manipulation system is the ability to simplify mathematical expressions. SymPy has dozens of functions to perform various kinds of simplification. There is also one general function called `simplify()` that attempts to apply all of these functions in an intelligent way to arrive at the simplest form of an expression.

In [57]:
x, y, z = symbols('x y z')

SymPy knows trigonometrical identities

In [58]:
simplify(sin(x)**2 + cos(x)**2)

1

Facotoring out commom factors in polynomials

In [59]:
simplify((x**3 + x**2 - x - 1)/(x**2 + 2*x + 1))

x - 1

And even understands identities from big formulas of statitistical distributions, like the gamma function.

In [60]:
simplify(gamma(x)/gamma(x - 2))

(x - 2)*(x - 1)

There is a caveat: "simplify" does not have a well-defined target, so the `simplify()` method might not be what you need.

In [72]:
expand((x + 1)**2)

x**2 + 2*x + 1

In [73]:
factor(x**2*z + 4*x*y*z + 4*y**2*z)

z*(x + 2*y)**2

Another powerfull tool is the `rewrite` method. The example below rewrites the tangent functions in terms of the sine function

In [76]:
tan(x).rewrite(sin)

2*sin(x)**2/sin(2*x)

And the factorial faction using the gamma function.

In [77]:
factorial(x).rewrite(gamma)

gamma(x + 1)

There are other types of simplifications as well, for powers, more elaborate trigonometric identities, logarithims, etc. Check out the documentations [here](https://docs.sympy.org/latest/tutorial/simplification.html).

---
## Calculus

In [93]:
x, y, z = symbols('x y z')

In [94]:
expr = exp(x**2 * y**2)
expr

exp(x**2*y**2)

The first steps we usually take in calculus usually envolves limits.

In [95]:
limit(sin(x)/x, x, 0)

1

and we learn that limits might be different from the right and from the left.

In [97]:
limit(1/x, x, 0, '+')

oo

In [98]:
limit(1/x, x, 0, '-')

-oo

You can take analytical derivatives with the `diff` method

In [82]:
diff(expr, x)

2*x*y**2*exp(x**2*y**2)

You can also take multiple derivatives by chaining input arguments

In [83]:
diff(expr, x, x, y)

4*y*(2*x**4*y**4 + 5*x**2*y**2 + 1)*exp(x**2*y**2)

The same idea can be used for integrals.

In [85]:
integrate(cos(x), x)

sin(x)

In [86]:
integrate(cos(x), y)

y*cos(x)

To evaluate an integral, pass variable and their limits as tuples

In [88]:
integrate(cos(x), (x, 0, pi/2))

1

In [89]:
integrate(cos(x), (y, 0, pi/2))

pi*cos(x)/2

You can even integrate to infinity

In [90]:
integrate(exp(-x**2 - y**2), (x, -oo, oo), (y, -oo, oo))

pi

In [91]:
integrate(exp(-x**2 - y**2), (x, -oo, oo), (y, -oo, oo))

pi

SymPy can also do taylor expansions aroundo a given point. The example below make a 4th degree approximation to the sine function aroun $x=0$.

In [99]:
sin(x).series(x, 0, 4)

x - x**3/6 + O(x**4)

And if you are an economist and do not care the $O(x^n)$ notation, you can remove it.

In [100]:
sin(x).series(x, 0, 4).removeO()

-x**3/6 + x

---
## Solvers

Writing equations in SymPy assumes $=0$ at the end, unless you are explicit about the right side.

In [102]:
Eq(x**2 -1)

Eq(x**2 - 1, 0)

In [103]:
Eq(x**2, 1)

Eq(x**2, 1)

In [105]:
Eq(x**2, x-1)

Eq(x**2, x - 1)

We can solve equations

In [114]:
expr = Eq(x**2 - 5*x + 6)
solve(expr, x)

[2, 3]

Sympy can even give elaborate answers to solutions. In this case, use `solveset` instead of `solve` 

In [115]:
solveset(x - x, x, domain=S.Reals)

Reals

In [116]:
solveset(x - x, x)

S.Complexes

In [117]:
solveset(sin(x) - 1, x)

ImageSet(Lambda(_n, 2*_n*pi + pi/2), Integers)

In [119]:
solveset(x**2 + 1, x)

{-I, I}

---
## Solving Differential Equations
The first step is to declare functions

In [120]:
f, g = symbols('f, g', cls=Function)

`f` and `g` are now undefined functions. We can call `f(x)`, and it will represent an unknown function.

In [121]:
f(x)

f(x)

In [123]:
f(x).diff()

Derivative(f(x), x)

In [124]:
diffeq = Eq(f(x).diff(x, x) - 2*f(x).diff(x) + f(x), sin(x))
diffeq

Eq(f(x) - 2*Derivative(f(x), x) + Derivative(f(x), (x, 2)), sin(x))

To solve the equation, use the `dsolve` method.

In [125]:
dsolve(diffeq, f(x))

Eq(f(x), (C1 + C2*x)*exp(x) + cos(x)/2)

Here, $C_{i}$ denotes arbitrary constants.

---
## Matrices
Matrix operations in SymPy are the same as in numpy. The only diffenrence is that now we can use symbols.

In [129]:
x, y = symbols('x, y')
M = Matrix([[1, 2, 3], [x, 3, 1], [4, y, 6]])
M

Matrix([
[1, 2, 3],
[x, 3, 1],
[4, y, 6]])

In [130]:
M.det()

3*x*y - 12*x - y - 10

Now take an example of a mtrix with incomplete rank

In [132]:
M = Matrix([[1, 1, 2], [2 ,1 , 3], [3 , 1, 4]])
M

Matrix([
[1, 1, 2],
[2, 1, 3],
[3, 1, 4]])

In [135]:
M.nullspace()  # or "kernel of the linear transformation"

[Matrix([
 [-1],
 [-1],
 [ 1]])]

In [136]:
M.columnspace()  # or "span of the linear transformation"

[Matrix([
 [1],
 [2],
 [3]]), Matrix([
 [1],
 [1],
 [1]])]

Notice that each output is a sympy object

In [140]:
display(M.diagonalize()[0])
display(M.diagonalize()[1])

Matrix([
[-1, -2*sqrt(11)/7 - 3/7, -3/7 + 2*sqrt(11)/7],
[-1,    2/7 - sqrt(11)/7,    2/7 + sqrt(11)/7],
[ 1,                   1,                   1]])

Matrix([
[0,            0,            0],
[0, 3 - sqrt(11),            0],
[0,            0, 3 + sqrt(11)]])

All of this was to get to the following example, the Jacobian Matrix:

In [144]:
from sympy.abc import rho, phi
X = Matrix([rho*cos(phi), rho*sin(phi), rho**2])
Y = Matrix([rho, phi])
X.jacobian(Y)

Matrix([
[cos(phi), -rho*sin(phi)],
[sin(phi),  rho*cos(phi)],
[   2*rho,             0]])

Just to grab a better view of what is happening here, let us build a simpler example

In [151]:
x, y, z = symbols('x, y, z')
eq1 = 2*x**2 + 3*y**3 - 2*z
eq2 = 4*x + y**2 - 2*z**3
eq3 = 3*x + 2*y - 3*z**4
M1 = Matrix([eq1, eq2, eq3])
M2 = Matrix([x, y])
M1.jacobian(M2)

Matrix([
[4*x, 9*y**2],
[  4,    2*y],
[  3,      2]])

This kind of structure will be perfect for DSGE models